In [1]:
import numpy as np
import scipy.stats as ss
import elfi

In [12]:
def fun(x, batch_size, random_state):
    return np.sum(x, keepdims=True, axis=1)

In [13]:
t1 = elfi.Prior(ss.multivariate_normal, [0, 0])
sim = elfi.Simulator(fun, t1, observed=np.array([[0]]))
d = elfi.Distance('euclidean', sim)

In [14]:
res = elfi.Rejection(sim, batch_size=10)